In [12]:
# Librerias
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats

## Funciones

## Importar Excels

### Mes 1

In [435]:
mes1 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-1.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes1=mes1.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes1=mes1.dropna(axis=0) #Elimina filas con NaN
tick = list(mes1.index)

weights = mes1['Weight (%)']
p1 = mes1['Price']
s1 = mes1['Shares']
a1 = p1*weights*s1 # Valor en dinero de cada ticker

v_etf1 = sum(a1) #Suma de los valores de las acciones
ETF1 =  v_etf1/17300000 #Valor del ETF1

mes1.head()

,Weight (%),Price,Shares
Ticker,,,
BA,9.32,176.86,1356276.0
UTX,8.63,112.21,1979767.0
LMT,7.66,267.60,736599.0
GD,6.77,187.20,930601.0
RTN,6.21,152.50,1047397.0


In [449]:
quandl.ApiConfig.api_key = "1YRXno4c2f3x5e96L3Dx" # Quandl API key
fechas = list(['2017-04-28','2017-05-31','2017-06-30','2017-07-31','2017-08-31','2017-09-29','2017-10-31','2017-11-30','2017-12-29','2018-01-31','2018-02-28','2018-03-27'])

In [450]:
def Download_QuandL(fechas = 'Dates_list ', tick ='Tickers of Stock'):
    dates = { 'gte': fechas[0], 'lte': fechas[1] }
    column = { 'columns': ['date', 'Open', 'Close']}
    data = quandl.get_table('WIKI/PRICES', qopts = column, ticker = tick, date = dates, paginate=True)# ticker = 'WIKI/AAPL' #'AAPL.US'\n",
    data = data.iloc[::-1]
    data = data.reset_index()
    data = data.drop(['None'],1)
    return data

In [451]:
def All_Data(fechas ='Dates_list ', tick ='Ticker'):
    d = {} # Crea un Diccionario Vacío\n",
    for i in range(len(tick)): # ciclo que descarga los datos de Quandl y los guarda como DataFrames en un Diccionario\n",
        nom = str(tick[i]) 
        d["_{0}".format(nom)] = Download_QuandL(fechas, tick[i])
        return d

In [452]:
def Rebalanceo(df, Com, Cap_I, R_R, n_a, we, op):
    #Esta funcion hace los rebalanceos de cada activo de forma diaria por una año 
    a = (df['close']/df['open'])-1
    P_se = list(df['open']*.99)
    add_s = list()
    Opp = list()
    comi = list()
    In_shares = np.floor((we*Cap_I)/df['close'][0])
    for i in range(len(df)):
        if a.iloc[i] < R_R:
            add_s.append(n_a)
            Opp.append('Compra')
            comi.append(Com*df['close'].iloc[i]*n_a)
            
        else:
            add_s.append(0)
            Opp.append('-')
            comi.append(0)
    df = df.join(pd.DataFrame(add_s, columns = ['# Added Shares']))
    df = df.join(pd.DataFrame(Opp, columns = ['Operacion']))
    df = df.join(pd.DataFrame(comi, columns = ['Comisión']))
    df = df.join(pd.DataFrame(P_se, columns = ['Precio de Señal']))
    Tot_Acciones = pd.DataFrame(list(df['# Added Shares'].cumsum()), columns = ['Total Shares'])+In_shares
    df = df.join(Tot_Acciones)
    Val_to_port = pd.DataFrame(list(df['close']*df['Total Shares']), columns = ['Value to ETF'])
    df = df.join(Val_to_port)
    
    frames = [op, df[df.Operacion == 'Compra']]
    op = pd.concat(frames)
    
    
    return df, op

In [453]:
def Clean_Dicc(dic = 'Diccionario a limpiar',sizes = 'Tamaño deseado del DF'):
    y = list(dic.keys())
    for h in y:
        if len(dic[h])!=sizes:
            del dic[h]
            
    return dic

In [454]:
Data = All_Data(fechas, tick)

In [455]:
d_full = Clean_Dicc(Data,248)

In [456]:
tick = list(d_full.keys())
d2 = {}
op = pd.DataFrame(columns=['date', 'open', 'close', '# Added Shares', 'Operacion', 'Comisión', 'Precio de Señal', 'Total Shares', 'Value to ETF'])
for j in range(len(tick)):
    tabla , op = Rebalanceo(d_full[tick[j]],0.0003,1000000,-0.01,10,weights[j], op)
    d2["Data{0}".format(j)] = tabla
op_2 = op.groupby('date')

In [457]:
len(Data)

0

### Mes 2

In [437]:
mes2 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-2.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes2=mes2.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes2=mes2.dropna(axis=0) #Elimina filas con NaN
tick = list(mes2.index)

weights = mes2['Weight (%)']
p2 = mes2['Price']
s2 = mes2['Shares']
a2 = p2*weights*s2 # Valor en dinero de cada ticker

v_etf2 = sum(a2) #Suma de los valores de las acciones
ETF2 =  v_etf2/17300000 #Valor del ETF1

mes2.head()

,Weight (%),Price,Shares
Ticker,,,
BA,9.57,184.83,1472774.0
UTX,9.00,118.99,2149824.0
LMT,7.58,269.45,799874.0
GD,6.89,193.79,1010539.0
RTN,6.21,155.21,1137366.0


### Mes 3

In [438]:
mes3 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-3.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes3=mes3.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes3=mes3.dropna(axis=0) #Elimina filas con NaN
tick = list(mes3.index)

weights = mes3['Weight (%)']
p3 = mes3['Price']
s3 = mes3['Shares']
a3 = p3*weights*s3 # Valor en dinero de cada ticker

v_etf3 = sum(a3) #Suma de los valores de las acciones
ETF3 =  v_etf3/17300000 #Valor del ETF1

mes3.head()

,Weight (%),Price,Shares
Ticker,,,
BA,9.48,187.63,1542834.0
UTX,8.94,121.28,2252087.0
LMT,7.71,281.13,837919.0
GD,7.04,203.25,1058608.0
RTN,6.40,164.01,1191467.0


### Mes 4

In [439]:
mes4 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-1.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes4=mes4.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes4=mes4.dropna(axis=0) #Elimina filas con NaN
tick = list(mes4.index)

weights = mes4['Weight (%)']
p4 = mes4['Price']
s4 = mes4['Shares']
a4 = p4*weights*s4 # Valor en dinero de cada ticker

v_etf4 = sum(a4) #Suma de los valores de las acciones
ETF4 =  v_etf4/17300000 #Valor del ETF1

mes4.head()

,Weight (%),Price,Shares
Ticker,,,
BA,9.32,176.86,1356276.0
UTX,8.63,112.21,1979767.0
LMT,7.66,267.60,736599.0
GD,6.77,187.20,930601.0
RTN,6.21,152.50,1047397.0


### Mes 5

In [440]:
mes5 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-5.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes5=mes5.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes5=mes5.dropna(axis=0) #Elimina filas con NaN
tick = list(mes5.index)

weights = mes5['Weight (%)']
p5 = mes5['Price']
s5 = mes5['Shares']
a5 = p5*weights*s5 # Valor en dinero de cada ticker

v_etf5 = sum(a5) #Suma de los valores de las acciones
ETF5 =  v_etf5/17300000 #Valor del ETF1

mes5.head()

,Weight (%),Price,Shares
Ticker,,,
BA,11.50,242.46,1650501.0
UTX,8.12,118.57,2383029.0
LMT,7.60,292.13,905103.0
GD,6.44,196.33,1140937.0
RTN,6.35,171.77,1285866.0


### Mes 6

In [441]:
mes6 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-6.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes6=mes6.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes6=mes6.dropna(axis=0) #Elimina filas con NaN
tick = list(mes6.index)

weights = mes6['Weight (%)']
p6 = mes6['Price']
s6 = mes6['Shares']
a6 = p6*weights*s6 # Valor en dinero de cada ticker

v_etf6 = sum(a1) #Suma de los valores de las acciones
ETF6 =  v_etf6/17300000 #Valor del ETF1

mes6.head()

,Weight (%),Price,Shares
Ticker,,,
BA,11.02,239.66,1846051.0
UTX,7.95,119.72,2665379.0
LMT,7.70,305.39,1012353.0
RTN,6.52,182.01,1438216.0
GD,6.40,201.35,1276137.0


### Mes 7

In [442]:
mes7 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-7.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes7=mes7.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes7=mes7.dropna(axis=0) #Elimina filas con NaN
tick = list(mes7.index)

weights = mes7['Weight (%)']
p7 = mes7['Price']
s7 = mes7['Shares']
a7 = p7*weights*s7 # Valor en dinero de cada ticker

v_etf7 = sum(a7) #Suma de los valores de las acciones
ETF7 =  v_etf7/17300000 #Valor del ETF1

mes7.head()

,Weight (%),Price,Shares
Ticker,,,
BA,10.14,254.21,1745134.0
UTX,7.87,116.08,2966161.0
LMT,7.37,310.29,1039522.0
GD,6.39,205.58,1360988.0
RTN,6.19,186.58,1452228.0


### Mes 8

In [443]:
mes8 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-8.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes8=mes8.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes8=mes8.dropna(axis=0) #Elimina filas con NaN
tick = list(mes8.index)

weights = mes8['Weight (%)']
p8 = mes8['Price']
s8 = mes8['Shares']
a8 = p8*weights*s8 # Valor en dinero de cada ticker

v_etf8 = sum(a8) #Suma de los valores de las acciones
ETF8 =  v_etf8/17300000 #Valor del ETF1

mes8.head()

,Weight (%),Price,Shares
Ticker,,,
BA,10.24,257.98,1807746.0
UTX,8.08,119.76,3072577.0
LMT,7.29,308.16,1076817.0
GD,6.28,202.98,1409815.0
NOC,6.24,295.53,961732.0


### Mes 9

In [444]:
mes9 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-9.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes9=mes9.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes9=mes9.dropna(axis=0) #Elimina filas con NaN
tick = list(mes9.index)

weights = mes9['Weight (%)']
p9 = mes9['Price']
s9 = mes9['Shares']
a9 = p9*weights*s9 # Valor en dinero de cada ticker

v_etf9 = sum(a9) #Suma de los valores de las acciones
ETF9 =  v_etf9/17300000 #Valor del ETF1

mes9.head()

,Weight (%),Price,Shares
Ticker,,,
BA,10.67,276.80,1886506.0
UTX,7.96,121.45,3206425.0
LMT,7.33,319.12,1123721.0
NOC,6.30,307.40,1003620.0
GD,6.23,207.16,1471239.0


### Mes 10

In [445]:
mes10 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-10.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes10=mes10.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes10=mes10.dropna(axis=0) #Elimina filas con NaN
tick = list(mes10.index)

weights = mes10['Weight (%)']
p10 = mes10['Price']
s10 = mes10['Shares']
a10 = p10*weights*s10 # Valor en dinero de cada ticker

v_etf10 = sum(a10) #Suma de los valores de las acciones
ETF10 =  v_etf10/17300000 #Valor del ETF1

mes10.head()

,Weight (%),Price,Shares
Ticker,,,
BA,10.86,294.91,1818068.0
UTX,8.24,127.57,3190353.0
LMT,7.50,321.05,1154158.0
GD,6.28,203.45,1525161.0
RTN,6.16,187.85,1618609.0


### Mes 11

In [446]:
mes11 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-11.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes11=mes11.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes11=mes11.dropna(axis=0) #Elimina filas con NaN
tick = list(mes11.index)

weights = mes11['Weight (%)']
p11 = mes11['Price']
s11 = mes11['Shares']
a11 = p11*weights*s11 # Valor en dinero de cada ticker

v_etf11 = sum(a11) #Suma de los valores de las acciones
ETF11 =  v_etf11/17300000 #Valor del ETF1

mes11.head()

,Weight (%),Price,Shares
Ticker,,,
BA,12.09,354.37,1953125.0
UTX,8.26,138.01,3427356.0
LMT,7.68,354.85,1239880.0
GD,6.37,222.48,1638456.0
RTN,6.34,208.94,1738846.0


### Mes 12

In [447]:
mes12 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-12.csv',
                   skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'], delim_whitespace=False, thousands=',')
mes12=mes12.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes12=mes12.dropna(axis=0) #Elimina filas con NaN
tick = list(mes12.index)

weights = mes12['Weight (%)']
p12 = mes12['Price']
s12 = mes12['Shares']
a12 = p12*weights*s12 # Valor en dinero de cada ticker

v_etf12 = sum(a12) #Suma de los valores de las acciones
ETF12 =  v_etf12/17300000 #Valor del ETF1

mes12.head()

,Weight (%),Price,Shares
Ticker,,,
BA,12.39,362.21,2004541.0
UTX,8.10,134.74,3524327.0
LMT,7.66,352.44,1273168.0
RTN,6.63,217.51,1785958.0
NOC,6.50,350.04,1088821.0


## Rendimientos

In [448]:
## Rendimiento de los ETF
Ren1 = np.log(ETF1/ETF12)
Ren2 = np.log(ETF2/ETF12)
Ren3 = np.log(ETF3/ETF12)
Ren4 = np.log(ETF4/ETF12)
Ren5 = np.log(ETF5/ETF12)
Ren6 = np.log(ETF6/ETF12)
Ren7 = np.log(ETF7/ETF12)
Ren8 = np.log(ETF8/ETF12)
Ren9 = np.log(ETF9/ETF12)
Ren10 = np.log(ETF10/ETF12)
Ren11 = np.log(ETF11/ETF12)

#DataFrame
r = np.array([Ren1,Ren2,Ren3,Ren4,Ren5,Ren6,Ren7,Ren8,Ren9,Ren10,Ren11])
series = ["Serie 1", "Serie 2", "Serie 3", "Serie 4", "Serie 5", "Serie 6", "Serie 7",
          "Serie 8", "Serie 9", "Serie 10", "Serie 11"]
tabla = pd.DataFrame(index=series,columns=['Rendimiento %'])
tabla.index.name = "Serie"
tabla['Rendimiento %'] = r
tabla

,Rendimiento %
Serie,
Serie 1,-0.951102
Serie 2,-0.814325
Serie 3,-0.734213
Serie 4,-0.951102
Serie 5,-0.580036
Serie 6,-0.951102
Serie 7,-0.418153
Serie 8,-0.364956
Serie 9,-0.270265
